In [ ]:
%load_ext autoreload
%autoreload 2

In [13]:

import json
import gzip
from itertools import islice
from collections import Counter, defaultdict
from operator import itemgetter
import pyarrow.parquet as pq
import pyarrow as pa

import pandas as pd
from tqdm.notebook import tqdm
import dill
import os
from pqdm.processes import pqdm
import re

from workers import *

import telepot
bot = telepot.Bot('436447719:AAGq_iK2hE3cPFhmL9Jh53g9HyVkw3SXZbg')
def qq(txt):
    bot.sendMessage(1114926, txt)

In [9]:
data_path = '..\\Data\\In\\'
submissions_path =  '..\\Submissions\\'

In [25]:
#df.to_pickle("df.pkl")

In [5]:
df = pd.read_pickle("df.pkl")

In [6]:
df.head()

,vacancy_id,name,description,area_id,creation_date,work_schedule,work_experience,compensation_from,compensation_to,currency,key_skills,employer,specializations,is_test,part,employment,spec_len,groups_of_spec,len_groups,html_tags
0,2700001,Прораб (Великий Устюг),<p><strong>Обязанности:</strong></p> <p>—взаим...,2,2019-06-14,fullDay,between3And6,NaN,NaN,NaN,[],72dc8d78bbec,None,True,10,full,NaN,None,NaN,"[p, strong, /strong, /p, p, /p, p, /p, p, /p, ..."
1,2700002,Лаборант электротехнической лаборатории,<p><strong>В связи с расширением Компании нам ...,1,2019-08-30,fullDay,between1And3,35000.0,45000.0,RUR,"[MS Office, Пользователь ПК, Деловое общение, ...",ba6dcbd0bcd1,None,True,10,full,NaN,None,NaN,"[p, strong, /strong, br /, strong, br /, /stro..."
2,2700003,Продавец-консультант ТЦ Радуга Парк,<strong>Обязанности:</strong> <ul> <li>консуль...,3,2019-06-17,fullDay,between1And3,25000.0,32000.0,RUR,"[Грамотная речь, Коммуникабельность, Активные ...",ddc67b017661,None,True,10,full,NaN,None,NaN,"[strong, /strong, ul, li, /li, li, /li, li, /l..."
3,2700004,Тестировщик в команду Saving products,<p>Мы - команда <strong>Saving products.</stro...,1,2019-09-25,fullDay,between1And3,NaN,NaN,NaN,[],a621c7f876ec,None,True,10,full,NaN,None,NaN,"[p, strong, /strong, br /, br /, br /, br /, e..."
4,2700005,Разработчик Python/django,<p>В небольшую дружную команду требуется опытн...,1,2019-09-03,fullDay,between3And6,70000.0,80000.0,RUR,[],845fe6266ab1,"[137, 221, 296]",False,10,full,3.0,[1],1.0,"[p, /p, p, /p, p, /p, p, /p, p, /p, p, /p, ul,..."


In [1]:
#df['html_tags'] = pqdm(df.description, extract_html, n_jobs=10)

In [7]:
df_train = df[df.is_test == False].copy()
del df

In [8]:
#df_train.to_parquet("df_tarin.pq")

In [ ]:
# if __name__ ==  '__main__': 
#     df['html_tags'] = parallelize_on_rows(df.description, extract_html) 

In [10]:
%%time
df_train= pd.read_parquet("df_tarin.pq")

CPU times: user 36.5 s, sys: 16.8 s, total: 53.2 s
Wall time: 1min 12s


In [11]:
df_train.columns

Index(['vacancy_id', 'name', 'description', 'area_id', 'creation_date',
       'work_schedule', 'work_experience', 'compensation_from',
       'compensation_to', 'currency', 'key_skills', 'employer',
       'specializations', 'is_test', 'part', 'employment', 'spec_len',
       'groups_of_spec', 'len_groups', 'html_tags', 'clear_text',
       'processed_text'],
      dtype='object')

In [15]:
%%time
if __name__ ==  '__main__':
    df_train['clear_text'] = parallelize_on_rows(df_train.description, remove_html) 

CPU times: user 16 s, sys: 17.1 s, total: 33 s
Wall time: 35 s


In [14]:
%%time
if __name__ ==  '__main__': 
    df_train['processed_text'] = parallelize_on_rows(df_train.clear_text, preprocess_text) 

CPU times: user 10.9 s, sys: 18 s, total: 28.8 s
Wall time: 18min 37s


In [15]:
qq('Precessing text done')

In [26]:
#test
df_train[df_train.vacancy_id == 2264555]['processed_text'].values

array(['требование уверенный пользователь высокий скорость печать внимательность грамотность ответственность эмоциональный уравновешенность активность стрессоустойчивость аккуратность исполнительность умение работать команда обязанность прием звонок выдача писемусловие трудоустройство официальный договор полный соц пакет график работа работа стабильный динамично развивающийся компания обучение стабильный профессиональный коллектив'],
      dtype=object)

In [27]:
%%time
df_train.to_parquet("df_tarin.pq")

CPU times: user 47.3 s, sys: 20.9 s, total: 1min 8s
Wall time: 2min 6s


In [28]:
df_text = df_train[['vacancy_id', 'name', 'specializations', 'key_skills', 'employer', 'groups_of_spec', 'processed_text']].copy()

In [29]:
del df_train

In [7]:
df_text = pd.read_parquet("df_text.pq")

In [74]:
df_text.head()

,vacancy_id,name,specializations,key_skills,employer,groups_of_spec,processed_text,text_list,key_list,name_list
4,2700005,разработчик python django,"[137, 221, 296]",,845fe6266ab1,[1],небольшой дружный команда требоваться опытный ...,"[небольшой, дружный, команда, требоваться, опы...",[],"[разработчик, python, django]"
5,2700006,торговый представитель,"[149, 242]",работа текущий база клиент мерчендайзинг работ...,23ead4a8f164,[17],обязанность компания дистрибьютор продукт пита...,"[обязанность, компания, дистрибьютор, продукт,...","[работа, текущий, база, клиент, мерчендайзинг,...","[торговый, представитель]"
6,2700007,менеджер оптовый продажа ткань,"[149, 196, 242, 324, 538]",b2b продажа активный продажа развитие продажа ...,9e3008e9f553,[17],стартекс крупный текстильный компания лидер ры...,"[стартекс, крупный, текстильный, компания, лид...","[b2b, продажа, активный, продажа, развитие, пр...","[менеджер, оптовый, продажа, ткань]"
8,2700010,врач клинический лабораторный диагностика бакт...,"[398, 537, 587]",,49f3280f53b6,[13],компания kdl клинико диагностический лаборатор...,"[компания, kdl, клинико, диагностический, лабо...",[],"[врач, клинический, лабораторный, диагностика,..."
12,2700015,швея портной,"[129, 157, 516]",,24e6806caa88,"[18, 11, 29]",требоваться швея лаборант навык закройщик прои...,"[требоваться, швея, лаборант, навык, закройщик...",[],"[швея, портной]"


In [12]:
%%time
if __name__ ==  '__main__':
    df_text['name'] = parallelize_on_rows(df_text.name, preprocess_text)

CPU times: user 1.38 s, sys: 1.44 s, total: 2.82 s
Wall time: 32.8 s


In [31]:
df_text['key_skills'] = df_text['key_skills'].apply(' '.join)

In [32]:
%%time
if __name__ ==  '__main__':
    df_text['key_skills'] = parallelize_on_rows(df_text.key_skills, preprocess_text)

CPU times: user 1.28 s, sys: 6.4 s, total: 7.68 s
Wall time: 39.9 s


In [33]:
from collections import Counter

In [79]:
df_text.drop(columns=['text_list'], inplace = True)

In [34]:
%%time
df_text['key_list'] = df_text['key_skills'].apply(lambda x: x.split(' '))
df_text['name_list'] = df_text['name'].apply(lambda x: x.split(' '))
# df_text['text_list'] = df_text['processed_text'].apply(lambda x: x.split(' '))

CPU times: user 6.71 s, sys: 3.17 s, total: 9.89 s
Wall time: 10.2 s


In [36]:
%%time
df_text.to_parquet("df_text.pq")

CPU times: user 11.7 s, sys: 2.99 s, total: 14.7 s
Wall time: 19.8 s


In [ ]:
%%time
df_text['text_list'] = df_text['processed_text'].apply(lambda x: x.split(' '))

In [ ]:
import functools
import operator
full_text = functools.reduce(operator.iconcat, df_text['text_list'].to_list(), [])

In [ ]:
word_count = Counter(full_text)

In [ ]:
word_count

In [ ]:
len(word_count)

In [ ]:
word_count.most_common(60)

In [ ]:
word_count.most_common()[:-100-1:-1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist([c for w, c in word_count.items() if c < 15000 and c > 10])

In [ ]:
qq('Ready')